### Implementación de un Modelo Transformer

In [1]:
import torch
import torch.nn as nn
import math
import copy

### 1. Codificación Posicional (Positional Encoding)

La codificación posicional añade información sobre la posición relativa o absoluta de los tokens en la secuencia. Las funciones sinusoidales fueron elegidas porque pueden permitir al modelo aprender a atender por posición relativa.

In [2]:
class PositionalEncoding(nn.Module):
    """
    Implementa la codificación posicional sinusoidal.
    """
    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1) # (max_len, 1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model)) # (d_model/2)
        
        pe = torch.zeros(max_len, 1, d_model) # (max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        # x se espera que sea (seq_len, batch_size, d_model)
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

### 2. Atención Multi-Cabeza (Multi-Head Attention)

Permite al modelo atender conjuntamente a información de diferentes subespacios
de representación en diferentes posiciones.

In [3]:
class MultiHeadAttention(nn.Module):
    """
    Implementa la atención multi-cabeza.
    """
    def __init__(self, d_model: int, num_heads: int, dropout: float = 0.1):
        super().__init__()
        assert d_model % num_heads == 0, "d_model debe ser divisible por num_heads"

        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads

        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)

        self.dropout = nn.Dropout(dropout)
        self.scale_factor = math.sqrt(self.d_k)

    def scaled_dot_product_attention(self, Q: torch.Tensor, K: torch.Tensor, V: torch.Tensor, mask: torch.Tensor = None) -> torch.Tensor:
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / self.scale_factor
        
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9) 
            
        attn_probs = torch.softmax(attn_scores, dim=-1)
        attn_probs = self.dropout(attn_probs)
        
        output = torch.matmul(attn_probs, V)
        return output

    def forward(self, query: torch.Tensor, key: torch.Tensor, value: torch.Tensor, mask: torch.Tensor = None) -> torch.Tensor:
        # query, key, value: (batch_size, seq_len, d_model)
        batch_size = query.size(0)

        Q = self.W_q(query)
        K = self.W_k(key)
        V = self.W_v(value)

        Q = Q.view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        K = K.view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        V = V.view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)

        context = self.scaled_dot_product_attention(Q, K, V, mask)
        
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)
        
        output = self.W_o(context)
        return output

### 3. Redes Feed-Forward (Position-wise Feed-Forward Networks)

Consiste en dos transformaciones lineales con una activación ReLU entre ellas, 
aplicadas independientemente a cada posición.

In [4]:
class PositionwiseFeedForward(nn.Module):
    """
    Implementa la red feed-forward (FFN) aplicada a cada posición.
    """
    def __init__(self, d_model: int, d_ff: int, dropout: float = 0.1):
        super().__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.linear1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear2(x)
        return x

### 4. Capa del Codificador (Encoder Layer)

Una capa del codificador consiste en una subcapa de atención multi-cabeza (self-attention) 
y una subcapa de red feed-forward. Se utilizan conexiones residuales y normalización de capa.

In [5]:
class EncoderLayer(nn.Module):
    """
    Una capa del codificador.
    """
    def __init__(self, d_model: int, num_heads: int, d_ff: int, dropout: float = 0.1):
        super().__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads, dropout)
        self.feed_forward = PositionwiseFeedForward(d_model, d_ff, dropout)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, src: torch.Tensor, src_mask: torch.Tensor = None) -> torch.Tensor:
        attn_output = self.self_attn(src, src, src, src_mask)
        src = self.norm1(src + self.dropout1(attn_output))

        ff_output = self.feed_forward(src)
        src = self.norm2(src + self.dropout2(ff_output))
        
        return src

### 5. Capa del Decodificador (Decoder Layer)

Similar a la capa del codificador, pero con una subcapa adicional que realiza 
atención multi-cabeza sobre la salida del codificador. La self-attention del 
decodificador está enmascarada.

In [6]:
class DecoderLayer(nn.Module):
    """
    Una capa del decodificador.
    """
    def __init__(self, d_model: int, num_heads: int, d_ff: int, dropout: float = 0.1):
        super().__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads, dropout)
        self.enc_dec_attn = MultiHeadAttention(d_model, num_heads, dropout)
        self.feed_forward = PositionwiseFeedForward(d_model, d_ff, dropout)
        
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.dropout3 = nn.Dropout(dropout)

    def forward(self, tgt: torch.Tensor, memory: torch.Tensor, 
                tgt_mask: torch.Tensor = None, memory_mask: torch.Tensor = None) -> torch.Tensor:
        attn_output = self.self_attn(tgt, tgt, tgt, tgt_mask)
        tgt = self.norm1(tgt + self.dropout1(attn_output))

        enc_dec_attn_output = self.enc_dec_attn(tgt, memory, memory, memory_mask)
        tgt = self.norm2(tgt + self.dropout2(enc_dec_attn_output))

        ff_output = self.feed_forward(tgt)
        tgt = self.norm3(tgt + self.dropout3(ff_output))
        
        return tgt

### 6. Codificador (Encoder)

El codificador es una pila de N capas EncoderLayer idénticas. Incluye la capa 
de embedding y la codificación posicional para la secuencia de entrada.

In [7]:
class Encoder(nn.Module):
    """
    El codificador es una pila de N capas EncoderLayer idénticas.
    """
    def __init__(self, num_layers: int, d_model: int, num_heads: int, d_ff: int,
                 input_vocab_size: int, max_seq_len: int, dropout: float = 0.1):
        super().__init__()
        self.d_model = d_model
        self.embedding = nn.Embedding(input_vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model, dropout, max_seq_len)
        self.layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) 
                                     for _ in range(num_layers)])
        self.dropout = nn.Dropout(dropout)

    def forward(self, src: torch.Tensor, src_mask: torch.Tensor = None) -> torch.Tensor:
        # src: (batch_size, src_seq_len)
        src_emb = self.embedding(src) * math.sqrt(self.d_model)
        # PositionalEncoding espera (seq_len, batch_size, d_model)
        src_emb = src_emb.transpose(0, 1) # (src_seq_len, batch_size, d_model)
        src_emb_pos = self.pos_encoder(src_emb)
        src_emb_pos = src_emb_pos.transpose(0, 1) # (batch_size, src_seq_len, d_model)
        
        output = self.dropout(src_emb_pos)

        for layer in self.layers:
            output = layer(output, src_mask)
            
        return output

### 7. Decodificador (Decoder)

El decodificador es también una pila de N capas DecoderLayer idénticas. 
Incluye la capa de embedding y la codificación posicional para la secuencia objetivo.

In [8]:
class Decoder(nn.Module):
    """
    El decodificador es también una pila de N capas DecoderLayer idénticas.
    """
    def __init__(self, num_layers: int, d_model: int, num_heads: int, d_ff: int,
                 output_vocab_size: int, max_seq_len: int, dropout: float = 0.1):
        super().__init__()
        self.d_model = d_model
        self.embedding = nn.Embedding(output_vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model, dropout, max_seq_len)
        self.layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout)
                                     for _ in range(num_layers)])
        self.dropout = nn.Dropout(dropout)

    def forward(self, tgt: torch.Tensor, memory: torch.Tensor, 
                tgt_mask: torch.Tensor = None, memory_mask: torch.Tensor = None) -> torch.Tensor:
        # tgt: (batch_size, tgt_seq_len)
        tgt_emb = self.embedding(tgt) * math.sqrt(self.d_model)
        # PositionalEncoding espera (seq_len, batch_size, d_model)
        tgt_emb = tgt_emb.transpose(0,1) # (tgt_seq_len, batch_size, d_model)
        tgt_emb_pos = self.pos_encoder(tgt_emb)
        tgt_emb_pos = tgt_emb_pos.transpose(0,1) # (batch_size, tgt_seq_len, d_model)
        
        output = self.dropout(tgt_emb_pos)

        for layer in self.layers:
            output = layer(output, memory, tgt_mask, memory_mask)
            
        return output

### 8. Modelo Transformer Completo

Une el codificador y el decodificador, y añade una capa lineal final para la 
predicción de tokens. También incluye métodos para crear las máscaras necesarias.

In [9]:
class Transformer(nn.Module):
    """
    Modelo Transformer completo.
    """
    def __init__(self, src_vocab_size: int, tgt_vocab_size: int, d_model: int, 
                 num_heads: int, num_encoder_layers: int, num_decoder_layers: int, 
                 d_ff: int, max_seq_len: int, dropout: float = 0.1, pad_idx: int = 0):
        super().__init__()
        self.pad_idx = pad_idx # Guardamos el índice de padding

        self.encoder = Encoder(num_encoder_layers, d_model, num_heads, d_ff, 
                               src_vocab_size, max_seq_len, dropout)
        self.decoder = Decoder(num_decoder_layers, d_model, num_heads, d_ff, 
                               tgt_vocab_size, max_seq_len, dropout)
        self.final_linear = nn.Linear(d_model, tgt_vocab_size)

        self._initialize_weights()

    def _initialize_weights(self):
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)

    def make_src_mask(self, src: torch.Tensor) -> torch.Tensor:
        # src: (batch_size, src_seq_len)
        # (batch_size, 1, 1, src_seq_len)
        src_mask = (src != self.pad_idx).unsqueeze(1).unsqueeze(2)
        return src_mask

    def make_tgt_mask(self, tgt: torch.Tensor) -> torch.Tensor:
        # tgt: (batch_size, tgt_seq_len)
        # (batch_size, 1, 1, tgt_seq_len)
        tgt_pad_mask = (tgt != self.pad_idx).unsqueeze(1).unsqueeze(2)
        
        tgt_len = tgt.size(1)
        # (1, 1, tgt_seq_len, tgt_seq_len)
        look_ahead_mask = torch.tril(torch.ones(tgt_len, tgt_len, device=tgt.device)).bool()
        look_ahead_mask = look_ahead_mask.unsqueeze(0).unsqueeze(0)
        
        # (batch_size, 1, tgt_seq_len, tgt_seq_len)
        tgt_mask = tgt_pad_mask & look_ahead_mask
        return tgt_mask

    def forward(self, src: torch.Tensor, tgt: torch.Tensor) -> torch.Tensor:
        src_mask = self.make_src_mask(src)
        tgt_mask = self.make_tgt_mask(tgt)
        memory_mask = src_mask # Para la atención encoder-decoder, la máscara de K,V es la de src

        enc_output = self.encoder(src, src_mask)
        dec_output = self.decoder(tgt, enc_output, tgt_mask, memory_mask)
        
        output = self.final_linear(dec_output)
        return output

### Explicación de Decisiones de Diseño

#### 1. Elección de Hiperparámetros (Ejemplos y Justificación):
*   **`d_model` (Dimensión del Modelo)**: e.g., 256 o 512. Es la dimensionalidad de los embeddings y las capas internas. Un valor mayor captura más información pero aumenta el costo computacional. Para un modelo 'sencillo', 256 podría ser un buen inicio.
*   **`num_heads` (Número de Cabezas de Atención)**: e.g., 8. `d_model` debe ser divisible por `num_heads`. Permite al modelo atender a diferentes partes de la información simultáneamente. `d_k = d_model / num_heads` es la dimensión de cada cabeza.
*   **`num_encoder_layers` / `num_decoder_layers` (Número de Capas)**: e.g., 2 a 3 (para un modelo sencillo). Más capas permiten aprender representaciones más complejas.
*   **`d_ff` (Dimensión de la Red Feed-Forward Interna)**: e.g., 1024 o 2048 (típicamente 4 * `d_model`). La expansión en esta capa permite al modelo aprender transformaciones más ricas.
*   **`dropout_rate` (Tasa de Dropout)**: e.g., 0.1. Ayuda a prevenir el sobreajuste.
*   **`max_seq_len` (Longitud Máxima de Secuencia)**: e.g., 100 o 512. Necesario para la codificación posicional precalculada.
*   **`vocab_size` (Tamaño del Vocabulario)**: Depende del corpus. Determina el tamaño de la capa de embedding y la capa lineal final.
*   **`pad_idx` (Índice de Padding)**: El índice usado para rellenar secuencias. Importante para las máscaras.

#### 2. Estructura de la Red:
*   **Embedding Layer**: Convierte tokens en vectores densos (`d_model`). Se escala por `sqrt(d_model)`.
*   **Positional Encoding**: Sumado a embeddings para información de posición (sinusoidal).
*   **Multi-Head Attention**:
    *   *Scaled Dot-Product Attention*: Calcula pesos de atención, escalados por `sqrt(d_k)`.
    *   *Múltiples Cabezas*: Proyecciones Q, K, V divididas, atención en paralelo, resultados concatenados.
*   **Add & Norm (Conexiones Residuales y Normalización de Capa)**:
    *   *Conexiones Residuales*: `x + Sublayer(x)`. Mitigan gradiente desvaneciente.
    *   *Layer Normalization*: Normaliza activaciones, estabiliza y acelera entrenamiento.
*   **Position-wise Feed-Forward Network (FFN)**: Dos capas lineales con ReLU, aplicadas por posición.
*   **Encoder**: Pila de `num_encoder_layers` EncoderLayer. Procesa entrada.
*   **Decoder**: Pila de `num_decoder_layers` DecoderLayer. Genera salida.
    *   *Masked Multi-Head Self-Attention*: Previene atender a tokens futuros en el decodificador.
    *   *Encoder-Decoder Attention*: Decodificador atiende a salida del codificador (Q de decoder, K,V de encoder).
*   **Capa Lineal Final + Softmax**: Proyecta salida del decodificador a `tgt_vocab_size`. Softmax (implícito en `CrossEntropyLoss`) para probabilidades.

#### 3. Funciones de Activación Utilizadas:
*   **ReLU (Rectified Linear Unit)**: `f(x) = max(0, x)`.
    *   *Ubicación*: En FFN.
    *   *Justificación*: Eficiente, mitiga gradiente desvaneciente, no linealidad.
*   **Softmax**: `softmax(z_i) = exp(z_i) / sum(exp(z_j))`.
    *   *Ubicación*:
        1.  Dentro de Scaled Dot-Product Attention (para pesos de atención).
        2.  Capa de salida final (para probabilidades sobre vocabulario, a menudo implícita en la función de pérdida).
    *   *Justificación*: Normaliza a distribución de probabilidad.

### Ejemplo de Uso Sencillo

In [10]:
# Hiperparámetros para el ejemplo
SRC_VOCAB_SIZE = 1000
TGT_VOCAB_SIZE = 1200
D_MODEL = 256 
NUM_HEADS = 8
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3
D_FF = 512 
MAX_SEQ_LEN = 80
DROPOUT = 0.1
PAD_IDX = 0 # Índice para el token de padding

# Fijar semilla para reproducibilidad
torch.manual_seed(0)

# Crear instancia del modelo Transformer
model = Transformer(src_vocab_size=SRC_VOCAB_SIZE,
                    tgt_vocab_size=TGT_VOCAB_SIZE,
                    d_model=D_MODEL,
                    num_heads=NUM_HEADS,
                    num_encoder_layers=NUM_ENCODER_LAYERS,
                    num_decoder_layers=NUM_DECODER_LAYERS,
                    d_ff=D_FF,
                    max_seq_len=MAX_SEQ_LEN,
                    dropout=DROPOUT,
                    pad_idx=PAD_IDX)

# Mover modelo a GPU si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")
model = model.to(device)
model.eval() # Poner en modo evaluación

# Crear datos de entrada ficticios
BATCH_SIZE = 2
SRC_SEQ_LEN = 10
TGT_SEQ_LEN = 12 

src_tokens = torch.randint(1, SRC_VOCAB_SIZE, (BATCH_SIZE, SRC_SEQ_LEN), device=device)
tgt_tokens = torch.randint(1, TGT_VOCAB_SIZE, (BATCH_SIZE, TGT_SEQ_LEN), device=device)

# Simular padding
if SRC_SEQ_LEN > 1:
    src_tokens[0, -1] = PAD_IDX
if TGT_SEQ_LEN > 1:
    tgt_tokens[0, -1] = PAD_IDX

print(f"\nForma de src_tokens: {src_tokens.shape}")
print(f"Forma de tgt_tokens: {tgt_tokens.shape}")

# Pasar los datos a través del modelo
with torch.no_grad():
    output_logits = model(src_tokens, tgt_tokens)

print(f"Forma de la salida (logits): {output_logits.shape}")

print("\n¡Implementación y ejemplo completados!")

Usando dispositivo: cpu

Forma de src_tokens: torch.Size([2, 10])
Forma de tgt_tokens: torch.Size([2, 12])
Forma de la salida (logits): torch.Size([2, 12, 1200])

¡Implementación y ejemplo completados!


Este es un Transformer 'sencillo' que cubre los componentes fundamentales. Para un uso práctico, se necesitaría un tokenizer, un conjunto de datos, un bucle de entrenamiento, una función de pérdida (e.g., `CrossEntropyLoss`), y un optimizador (e.g., Adam).